# 📚 RAG 실전 활용 예제

실무에서 바로 사용할 수 있는 RAG 활용법을 배워봅시다.

## 🔧 준비: RAG 엔진 초기화

In [1]:
import os
from dotenv import load_dotenv
from ontology_rag.core.rag_engine import RAGEngine
from ontology_rag.embeddings.client import EmbeddingClient
from ontology_rag.storage.vector_store import VectorStore

load_dotenv()
LLM_BASE_URL = os.getenv("LLM_BASE_URL", "http://localhost:11434")
LLM_MODEL = os.getenv("LLM_MODEL", "qwen2.5vl:72b")

embedder = EmbeddingClient(base_url=LLM_BASE_URL)
store = VectorStore()
rag = RAGEngine(
    llm_base_url=LLM_BASE_URL,
    llm_model=LLM_MODEL,
    embedding_client=embedder,
    vector_store=store,
)

print("✅ RAG 엔진 준비 완료")

✅ RAG 엔진 준비 완료


## 1️⃣ 텍스트 파일에서 문서 불러오기

실무에서는 보통 파일로 문서를 관리합니다.

In [4]:
from pathlib import Path

# 파일에서 문서 읽기
file_path = "../data/my_document.txt"

if Path(file_path).exists():
    content = Path(file_path).read_text(encoding="utf-8")
    
    # 문단별로 나누기
    chunks = [chunk.strip() for chunk in content.split("\n\n") if chunk.strip()]
    
    rag.add_documents(chunks)
    print(f"✅ {len(chunks)}개 문서 추가 완료")
    print(f"\n첫 번째 문서 미리보기:\n{chunks[0][:100]}...")
else:
    print("⚠️ 파일이 없습니다. data/my_document.txt 파일을 만들어주세요.")

✅ 4개 문서 추가 완료

첫 번째 문서 미리보기:
What is Ontology?...


## 2️⃣ 검색 결과 확인하기 (디버그 모드)

`debug=True`로 어떤 문서가 검색되는지 확인할 수 있습니다.

In [10]:
question = "온톨로지가 뭐야?"

print("🔍 검색 중...\n")
answer = rag.query(question, top_k=6, debug=True)

print(f"\n💬 최종 답변:\n{answer}")

🔍 검색 중...

Search results: {'ids': [['doc_3', 'doc_1', 'doc_0', 'doc_2']], 'embeddings': None, 'documents': [['1. Domain Ontology: Represents concepts specific to a particular domain\n2. Upper Ontology: Describes general concepts that are the same across all domains\n3. Task Ontology: Describes concepts related to a specific task or activity', 'PFCT의 주요 제품:\n    1. 크플(cple): 온투금융 플랫폼, 중금리 대출과 투자 연결\n    2. 에어팩(AIRPACK): AI 기반 신용 리스크 관리 B2B 솔루션\n    3. 비은행권 최고의 AI 신용평가 기술력 보유', "PFCT(PFC Technologies, 피에프씨테크놀로지스)는 대한민국의 선도적인 핀테크 기업입니다.\n    온라인투자연계금융 플랫폼 '크플(cple)'을 운영하며, 대출자와 투자자를 연결합니다.", 'PFCT 주요 성과:\n    - 대한민국 온투금융사 1호 등록\n    - 2022년 Forbes Asia 100 to Watch 선정\n    - 2024년 3월 피플펀드컴퍼니에서 피에프씨테크놀로지스로 사명 변경\n    - 투자자에게 연 10% 내외의 안정적인 수익 제공']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None, None, None, None]], 'distances': [[203.50357055664062, 423.38555908203125, 434.1642761230469, 477.80059814453125]]}
Found 4 documents
Doc 1: 

## 3️⃣ top_k 값 비교하기

`top_k`는 검색할 문서 개수입니다. 값에 따라 답변이 어떻게 달라지는지 확인해봅시다.

In [7]:
question = "온톨로지의 종류는?"

for k in [1, 2, 3]:
    print(f"\n{'='*80}")
    print(f"📊 top_k = {k}")
    print(f"{'='*80}")
    
    answer = rag.query(question, top_k=k)
    print(f"답변: {answer}")


📊 top_k = 1
답변: 문서에서 해당 정보를 찾을 수 없습니다.

📊 top_k = 2
답변: 문서에서 해당 정보를 찾을 수 없습니다.

📊 top_k = 3
답변: 문서에서 해당 정보를 찾을 수 없습니다.


**관찰 포인트:**
- `top_k=1`: 가장 관련 높은 문서 1개만 사용 → 빠르지만 정보 부족 가능
- `top_k=3`: 여러 문서 참고 → 더 풍부한 답변, 하지만 느림
- `top_k=6+`: 관련 없는 문서까지 포함될 수 있음 → 답변 품질 저하
- **권장**: 일반적으로 2-3이 적당

**⚠️ 주의**: k 값이 너무 크면 관련 없는 문서가 섞여서 LLM이 혼란스러워합니다!

## 4️⃣ 저장된 문서 확인하기

In [ ]:
# 전체 문서 개수
count = store.count()
print(f"📚 저장된 문서: {count}개\n")

# 문서 목록 보기
results = store.get_all()

for i, (doc_id, doc) in enumerate(zip(results['ids'], results['documents']), 1):
    preview = doc[:80] + "..." if len(doc) > 80 else doc
    print(f"{i}. [{doc_id}]")
    print(f"   {preview}\n")

## 5️⃣ 문서 초기화 (다시 시작하기)

In [ ]:
# 주의: 모든 문서가 삭제됩니다!
# store.clear()
# print("🗑️ 모든 문서 삭제 완료")

print("⚠️ 위 코드의 주석을 해제하면 모든 문서가 삭제됩니다.")

## 6️⃣ 실무 시나리오: 금융 상품 Q&A

In [ ]:
# 금융 상품 정보 추가
product_docs = [
    """크플 투자 상품 안내
    - 최소 투자금액: 10만원
    - 예상 수익률: 연 8-12%
    - 투자 기간: 3개월, 6개월, 12개월 선택 가능
    - 중도 해지: 가능 (수수료 발생)""",
    
    """크플 대출 상품 안내
    - 대출 한도: 최대 3,000만원
    - 금리: 연 8-15% (신용등급에 따라 차등)
    - 상환 방식: 원리금균등분할상환, 만기일시상환
    - 중도상환 수수료: 없음""",
    
    """크플 회원 가입 방법
    1. 크플 앱 다운로드 또는 웹사이트 접속
    2. 본인 인증 (휴대폰, 신용카드)
    3. 계좌 연결
    4. 투자자 적합성 테스트 완료
    5. 가입 완료 (약 5분 소요)"""
]

# 기존 문서 초기화 후 새 문서 추가
store.clear()
rag.add_documents(product_docs)
print(f"✅ {len(product_docs)}개 상품 정보 추가 완료\n")

In [ ]:
# 고객 질문 시뮬레이션
customer_questions = [
    "최소 투자금액이 얼마야?",
    "대출 금리는 어떻게 되나요?",
    "회원가입은 어떻게 하나요?",
    "중도 해지 가능한가요?"
]

print("💬 고객 Q&A 시뮬레이션\n")
print("="*80)

for q in customer_questions:
    answer = rag.query(q, top_k=2)
    print(f"\n고객: {q}")
    print(f"상담원: {answer}")
    print("-"*80)

## 7️⃣ 답변이 이상할 때 체크리스트

### ❌ 문제: "정보를 찾을 수 없습니다"
**해결:**
1. 문서가 제대로 추가되었는지 확인: `store.count()`
2. 질문과 문서의 표현이 너무 다른지 확인
3. `top_k` 값을 늘려보기

### ❌ 문제: 엉뚱한 답변
**해결:**
1. `debug=True`로 어떤 문서가 검색되는지 확인
2. 문서를 더 작은 단위로 나누기
3. 관련 없는 문서 삭제

### ❌ 문제: 너무 느림
**해결:**
1. `top_k` 값 줄이기 (3 → 2 → 1)
2. 문서 개수 줄이기
3. 더 작은 LLM 모델 사용

## 🎯 다음 단계

1. **여러분의 문서로 실험해보세요**
   - 회사 규정, 매뉴얼, FAQ 등
   
2. **CLI로 사용하기**
   ```bash
   ontology-rag add data/my_doc.txt
   ontology-rag query "질문"
   ```

3. **더 배우기**
   - 프롬프트 엔지니어링
   - 하이브리드 검색
   - 멀티모달 RAG (이미지, 표 포함)